# Add pmi-able informations


In [1]:
import os,sys,inspect
import pandas as pd
import numpy as np
import importlib
import sys
from utils_xo import *
importlib.reload(sys.modules["utils_xo"])
from utils_xo import *
pd.set_option('display.max_colwidth', None)
# set data directory
data_dir = r"../../../data/french/"
import xml.etree.ElementTree as ET
import re

In [30]:
pmi_able = xo_read_json(data_dir+"pmi_frwac_alt_corrected.json")
no_pmi = xo_read_json(data_dir+"winofr_final_v3.json")
no_pmi
pmiable_schemas = []
for it in no_pmi:
    fr_id = it["french_schema_no"]
    for x in pmi_able:
        if int(x["schema"]) == fr_id:
            pmiable_schemas.append(fr_id)
# -29 82 96  91 - 3 schemas  = 88*2 176 items

In [31]:
len(pmiable_schemas) # from 178 to 176

176

In [38]:
#! construct lists of r0 r1 special alternate pmi_able
#* 1 schema = 2 items = 4 pairs, 2 pairs for each item
special_pairs = []
alternate_pairs = []

for i in pmiable_schemas:
    for it in pmi_able:
        if int(it["schema"]) == i:
            special_pairs.append((i,it["special"]+"|"+it["r0"], it["special"]+"|"+it["r1"]))
            alternate_pairs.append((i,it["alternate"]+"|"+it["r0"], it["alternate"]+"|"+it["r1"]))

In [132]:
# special_pairs

In [131]:
all_pairs = []
for x,y in zip(special_pairs,alternate_pairs):
    all_pairs.append(x)
    all_pairs.append(y)
# all_pairs

In [130]:
half_pairs = []
len_pairs = len(all_pairs)
len_pairs
first = list(range(0,len_pairs,4))
second = list(range(1,len_pairs,4))
for x,y in zip(first,second):
    half_pairs.append(all_pairs[x])
    half_pairs.append(all_pairs[y])
# half_pairs
# 0,4,8
# 1,5,9

In [55]:
from collections import defaultdict
temp = defaultdict(list)
for i,it in enumerate(no_pmi):
    # get schema id
    schema_id = it["french_schema_no"]
    # get pairs in all_pairs
    for j in half_pairs:
        if j[0] == schema_id:
            temp[schema_id].append(j[1:])

In [129]:
# temp

In [71]:
log = 1
for i,it in enumerate(no_pmi):
    schema = it["french_schema_no"]
    if schema in temp and log == 1:
        it["pmi_able"] = 1
        it["question_right"] = temp[schema][0][0]
        it["question_wrong"] = temp[schema][0][1]
        log = 2
    elif schema in temp and log != 1:
        it["pmi_able"] = 1
        it["question_right"] = temp[schema][1][0]
        it["question_wrong"] = temp[schema][1][1]
        log = 1

In [78]:
for it in no_pmi:
    # print(it["pmi_able"])
    if it["pmi_able"] == 0:
        it["question_right"] = ""
        it["question_wrong"] = ""

In [125]:
import re
#! make hypothesis
for it in no_pmi:
    try:
        it["hypothesis"] = re.split(r",|;|\?",it["masked_text"])[1].replace("<mask>",it["correct_answer_head"]).strip()
    except:
        it["hypothesis"] = it["masked_text"].replace("<mask>",it["correct_answer_head"]).strip()
for it in no_pmi:
    try:
        it["masked_hypothesis"] = re.split(r",|;|\?",it["masked_text"])[1].strip()
    except:
        it["masked_hypothesis"] = it["masked_text"].replace("<mask>",it["correct_answer_head"]).strip()

In [126]:
no_pmi = sorted(no_pmi,key= lambda x: x["id"])
# remove the no_contexte
xo_write_json_fromlist(data_dir+"v4.json",no_pmi)

In [127]:
df = pd.read_json(data_dir+"v4.json")
df.drop(columns=["no_contexte"],inplace=True)

In [128]:
df
# xo_write_json(data_dir+"v5.json",df)
#! the manuel correction phase going on

In [ ]:
#!make hypothese and masked hypothese first letter uppercase
#! start here  2021-03-12, ignore above